In [1]:
%matplotlib inline
import pylab as plt
plt.style.use('fivethirtyeight')

import pandas as pd
import json
from copy import copy

In [2]:
path = '../Chapter07/all_battles_parsed.json'
with open(path, 'r') as f:
    battles = json.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '../Chapter07/all_battles_parsed.json'

In [24]:
from pandas.io.json import json_normalize

### Flatten all battles, campains, etc.

In [25]:
def _flatten_battles(battles, root=None):
    buttles_to_run = copy(battles)
    records = []
    
    for name, data in battles.items():
        if 'children' in data:
            children = data.pop('children')
            records.extend(_flatten_battles(children, root=name))
        else:
            data['level'] = 100
    
        data['name'] = name
        data['parent'] = root
        records.append(data)

    return records    

In [26]:
records = {k: _flatten_battles(v, root=k) for k, v in battles.items()} # fronts

In [27]:
records = {k: pd.DataFrame(json_normalize(v)) for k, v in records.items()}

### Store as CSV

In [28]:
for front, data in records.items():
    data.to_csv(f'./data/{front}.csv', index=None)